### Deutsch-Jozsa algorithm

The [Deutsch-Jozsa algorithm](https://en.wikipedia.org/wiki/Deutsch%E2%80%93Jozsa_algorithm) is the classical demonstration of the quantum parallelism: the application of a function to all the quantum states in a single step. 

The main idea is to find if a binary function:
$$f:Z_{2^n} \to Z_2$$
that is known to be *constant* (for every x the result is equal, or 0 or 1) or *balanced* (produces the same number of 0s and 1s), is constant or balanced.

To compute it, the value of the function is encoded in the phase of one state as:
$$U_f(|x>) = (-1)^{f(x)} |x>$$

Where $$U_f$$ is named as *oracle*. Here, you will use a simple case:
$$f(x) = x\%2 $$

so, using the integer or binary representation of one n-qubit state

$$f(|00...00\rangle)=f(|0\rangle)= +|0\rangle$$
$$f(|00...01\rangle)=f(|1\rangle)=-|1\rangle$$
$$f(|00...10\rangle)=f(|2\rangle)= +|2\rangle$$
$$f(|00...11\rangle)=f(|3\rangle)=-|3\rangle$$
$$etc.$$

The algorithm has four steps:

1. Starting from a $|0\rangle$ state of *n* qubits, apply a Walsh-Hadamard transformation to create the state $|\Phi\rangle=\frac{1}{\sqrt(2^n)}\sum_{i=0}^{2^n-1}|i\rangle$
2. Apply the desired oracle.
3. Uncompute operation 1
4. Measure all qubits and compare with state |0>


In [1]:
import projectq
from projectq.cengines import MainEngine
from projectq.ops import H, Z, X, Measure, All

Start the Engine and allocate an even number of qubits (by default, 4)

In [2]:
eng=MainEngine()
qreg=eng.allocate_qureg(4)


Create a superposition of all the posible states with the number of allocated qubits, applying a Walsh-Hadamard operator. The cheat() method will print the amplitudes for all the states: |0000>,|0001>,etc.

In [3]:
All(H)|qreg
eng.flush()
eng.backend.cheat()

({0: 0, 1: 1, 2: 2, 3: 3},
 [(0.24999999999999992+0j),
  (0.24999999999999992+0j),
  (0.24999999999999992+0j),
  (0.24999999999999992+0j),
  (0.24999999999999992+0j),
  (0.24999999999999992+0j),
  (0.24999999999999992+0j),
  (0.24999999999999992+0j),
  (0.24999999999999992+0j),
  (0.24999999999999992+0j),
  (0.24999999999999992+0j),
  (0.24999999999999992+0j),
  (0.24999999999999992+0j),
  (0.24999999999999992+0j),
  (0.24999999999999992+0j),
  (0.24999999999999992+0j)])

To create the Oracle for the function, it is enought to compute Z applied to first qubit (qubit 0 in your quantum register), this means, for *n* qubits:

$$ I^{\otimes n-1}\otimes Z $$

For 2 qubits this operator, in matrix form, looks as:

$$ I \otimes Z = \begin{bmatrix}1&0\\0&1\end{bmatrix}\otimes\begin{bmatrix}1&0\\0&-1\end{bmatrix}=
\begin{bmatrix}1&0&0&0\\0&-1&0&0\\0&0&1&0\\0&0&0&-1\end{bmatrix}$$

Applying it to the generated state: 
$$|\Phi\rangle = \frac{1}{2}\begin{bmatrix}1\\1\\1\\1\\\end{bmatrix}=\frac{1}{2}(|0\rangle+|1\rangle+|2\rangle+|3\rangle)$$

yields:

$$|\phi\rangle = \frac{1}{2}\begin{bmatrix}1\\-1\\1\\-1\\\end{bmatrix}$$

or, using the integer representation:

$$|\phi\rangle = \frac{1}{2}(|0\rangle-|1\rangle+|2\rangle-|3\rangle)$$



In [4]:
Z|qreg[0]
#
#Other oracles you can check
#
#X|qreg[0] #Constant
#All(X)|qreg #Constant
#All(Z)|qreg #balanced
eng.flush()
eng.backend.cheat()

({0: 0, 1: 1, 2: 2, 3: 3},
 [(0.24999999999999992+0j),
  (-0.24999999999999992+0j),
  (0.24999999999999992+0j),
  (-0.24999999999999992+0j),
  (0.24999999999999992+0j),
  (-0.24999999999999992+0j),
  (0.24999999999999992+0j),
  (-0.24999999999999992+0j),
  (0.24999999999999992+0j),
  (-0.24999999999999992+0j),
  (0.24999999999999992+0j),
  (-0.24999999999999992+0j),
  (0.24999999999999992+0j),
  (-0.24999999999999992+0j),
  (0.24999999999999992+0j),
  (-0.24999999999999992+0j)])

Uncompute the superposition

In [5]:
All(H)|qreg
eng.flush()
eng.backend.cheat()

({0: 0, 1: 1, 2: 2, 3: 3},
 [0j,
  (0.9999999999999993+0j),
  0j,
  0j,
  0j,
  0j,
  0j,
  0j,
  0j,
  0j,
  0j,
  0j,
  0j,
  0j,
  0j,
  0j])

Measure the results. If it is **constant**, the result must be |0000>=|0>. If it is **balanced**, the result is different of |0>

In [6]:
All(Measure) | qreg

In [7]:
eng.flush()
output=0
for i in qreg:
    output+=int(i)
balanced=output&0
print("The result is %s"%("balanced" if output else "constant"))

The result is balanced


If you want to check that the result is 0 if is constant, substitute the gate Z in the oracle by X.
If you want to test with another balanced function, substitute Z by All(Z), but substitute also qreg[0] by qreg